In [ ]:
import pandas as pd
import numpy as np

In [ ]:
adj_data = pd.read_excel("Adjectives.xlsx")
adj_data = adj_data.dropna()
cmt_data = pd.read_excel("Womens Clothing E-Commerce Reviews (1).xlsx")
cmt_data = cmt_data.dropna()

In [ ]:
pos_adj = list(adj_data['Positive adjectives'])
neg_adj = list(adj_data['Negative Adjectives'])
neu_adj = list(adj_data['Neutral adjectives'])

In [ ]:
color = ['color', 'print', 'pattern']
size = ['size', 'petit', 'petite', 'shape', 'height', 'bust', 'fit', 'sizing', 'neckline', 'fitting', 'length']
material = ['texture', 'denim', 'wool', 'cotton', 'linen', 'material']
comfort = ['comfort', 'comfortable']
longevity = ['durable', 'lasting', "age"]

categories_dict = {'Color': color, "Size": size, "Material": material, "Comfort": comfort,
                   "Longevity": longevity}

stop_words = ['a', 'an', 'the', 'this', 'these', 'is', 'are', 'in', 'on', 'at', 'it', "it's"]

neg_verbs = ["isn't", 'not', "ain't", "aren't", "hasn't", "haven't"]

cmt_data = cmt_data.assign(Size="", Material="", Longevity = "", Color="", General="", Comfort="")

In [ ]:
del cmt_data['Comment Type']

In [ ]:
reviews = cmt_data['Review Text'].str.strip()

import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = str(text).lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
cmt_data['Review Text'] = normalize_texts(cmt_data['Review Text'])

In [ ]:
rev = pd.DataFrame(cmt_data['Review Text'])
rev["Review Text"] = rev["Review Text"].replace(r'\s+', ' ', regex=True)
cmt_data["Review Text"] = rev["Review Text"]

In [ ]:
for index, row in cmt_data.iterrows():

  split_review = row["Review Text"].split()    
  # this can also be done before (in the cleaning)

  review_no_stopwords = [ r_word for r_word in split_review if r_word not in stop_words]

  for i, rns_word in enumerate(review_no_stopwords):

    for category_name, category_list in categories_dict.items():
      
      
      if row[category_name] == '':
        text_eval = ''
        if i != len(review_no_stopwords) - 1:
          if review_no_stopwords[i + 1] in category_list:
            if rns_word in pos_adj:
              text_eval ='POS'
            if rns_word in neg_adj:
              text_eval ='NEG'
            if rns_word in neu_adj:
              text_eval ='NEU'

        if i != 0:
          if review_no_stopwords[i - 1] in category_list:
            if rns_word in pos_adj:
              text_eval ='POS'
            if rns_word in neg_adj:
              text_eval ='NEG'
            if rns_word in neu_adj:
              text_eval ='NEU'

        if i > 1:
          if (review_no_stopwords[i - 1] in neg_verbs and 
              review_no_stopwords[i - 2] in category_list):
            if rns_word in pos_adj:
              text_eval ='NEG'
            if rns_word in neg_adj:
              text_eval ='POS'
          
        cmt_data.at[index, category_name] = text_eval

In [ ]:
cmt_data.head
cmt_data.to_excel("Output Womens Clothing E-Commerce Reviews.xlsx")  